In [13]:
from keras.layers import Input, Dense, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.callbacks import TensorBoard
from tensorboard.plugins.hparams import api as hp
import tensorflow as tf
import numpy as np
import pickle
from datetime import datetime
from contextlib import redirect_stdout
import uuid

In [14]:
L = 4
FEATURE_MAP_START = 16

In [15]:
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([15, 50]))
HP_N_LAYERS = hp.HParam('n_layers', hp.Discrete([2, 3]))
HP_FEATURE_MAP_STEP = hp.HParam('feature_map_step', hp.Discrete([2, 8, 16]))

# Notes:
* Add `Input` argument `channels_first` when using Z3 data and 
  change the input dimensions.
* You will have to manualy add periodic edges for periodic padding before real runn
* **requires** running the transform notebook to get the data into a format for the auto encoder

In [16]:
with open('pickle_data.pkl', 'rb') as f:
    all_data = pickle.load(f)

all_data = all_data.astype('float32') / 5.0
n_records = len(all_data)
x_train = all_data[: n_records - int(n_records/4)]
x_test = all_data[int(n_records/4) :]
x_train = np.reshape(x_train, (len(x_train), L * 2, L * 2, 1))
x_test = np.reshape(x_test, (len(x_test), L * 2, L * 2, 1))

In [17]:
with tf.summary.create_file_writer('/tmp/hp_autoencoder').as_default():
    hp.hparams_config(
        hparams=[HP_BATCH_SIZE, HP_FEATURE_MAP_STEP],
        metrics=[hp.Metric('binary_crossentropy', display_name='Loss')]
    )

In [18]:
for i in range(4 - 1):
    print(FEATURE_MAP_START + (i + 1) * 8)

24
32
40


In [19]:
for i in range(4 - 1):
    print(32 - (i + 1) * 8)

24
16
8


In [20]:
def train_test_model(hparams):
    current_run_id = uuid.uuid4().hex
    
    input_obj = Input(shape=(L*2, L*2, 1))
    x = Conv2D(FEATURE_MAP_START, (3, 3), strides=2, activation='relu', padding='same', use_bias=True)(input_obj)
    for i in range(hparams[HP_N_LAYERS] - 1):
        fm = FEATURE_MAP_START + (i + 1) * hparams[HP_FEATURE_MAP_STEP]
        x = Conv2D(fm, (3, 3), strides=2, activation='relu', padding='same', use_bias=True)(x)
        
    max_fm = fm
    for i in range(hparams[HP_N_LAYERS] - 1):
        fm = max_fm - (i + 1) * hparams[HP_FEATURE_MAP_STEP]
        x = Conv2DTranspose(fm, (3, 3), strides=2, activation='relu', padding='same', use_bias=True)(x)
    
    decoded = Conv2DTranspose(1, (3, 3), strides=2, activation='relu', padding='same', use_bias=True)(x)

    autoencoder = Model(input_obj, decoded)
    #autoencoder.summary()
    autoencoder.compile(
        optimizer='adadelta',
        loss='binary_crossentropy',
        metrics=['crossentropy']
    )

    log_dir='/tmp/hp_autoencoder/{}_{}'.format(datetime.now(), current_run_id)
    autoencoder.fit(
        x_train, x_train,
        epochs=50,
        batch_size=hparams[HP_BATCH_SIZE],
        shuffle=True,
        validation_data=(x_test, x_test),
        callbacks=[
            TensorBoard(log_dir=log_dir),
            hp.KerasCallback(log_dir, hparams)
        ]
    )
    _, binary_crossentropy = autoencoder.evaluate(x_test, x_test)
    return binary_crossentropy

In [21]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        loss = train_test_model(hparams)
        tf.summary.scalar('binary_crossentropy', loss, step=1)

In [ ]:
c = 0
for batch_size in HP_BATCH_SIZE.domain.values:
    for n_layers in HP_N_LAYERS.domain.values:
        for f_map_step in HP_FEATURE_MAP_STEP.domain.values:
            hparams = {
                HP_BATCH_SIZE: batch_size,
                HP_N_LAYERS: n_layers,
                HP_FEATURE_MAP_STEP: f_map_step
            }
            c += 1
            run_name = "run-%d" % c
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)


--- Starting trial: run-1
{'batch_size': 15, 'n_layers': 2, 'feature_map_step': 2}
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 8, 8, 1)]         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 4, 4, 16)          160       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 2, 2, 18)          2610      
_________________________________________________________________
conv2d_transpose_10 (Conv2DT (None, 4, 4, 16)          2608      
_________________________________________________________________
conv2d_transpose_11 (Conv2DT (None, 8, 8, 1)           145       
Total params: 5,523
Trainable params: 5,523
Non-trainable params: 0
_________________________________________________________________
Epoch 1/120
500/500 [===================

In [ ]:
x_train.shape